In [1]:
# Suppressing Warnings
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [2]:
victoria_secret = pd.read_csv('Dataset/Intermediate/victoria_secret_products.csv')
victoria_secret.head()

,Unnamed: 0,product_name,product_category,description,product_category_group,mrp,price,size_popularity,color_popularity,%discount
0,0,Allover Lace from Cotton Lingerie NEW! Dotted ...,Dotted Mesh Bikini Panty,Dotted mesh in the ultimate everyday shape mak...,panties,10.500000,10.500000,0.126413,0.154872,0.00
1,1,Allover Lace from Cotton Lingerie NEW! Dotted ...,Dotted Mesh Thong Panty,Dotted mesh in a sexy little shape makes this ...,panties,10.500000,10.500000,0.126413,0.154872,0.00
2,2,Body by Victoria Cheekini Panty,Cheekini Panty,"The comfiest way to show a little cheek peek, ...",panties,14.500000,7.142542,0.754805,0.163567,50.74
3,3,Body by Victoria Daisy Lace Slip,Daisy Lace Slip,What dreams are made of: this little slip in f...,slip,52.000000,34.990000,0.126413,0.155092,32.71
4,4,Body by Victoria Demi Bra,Demi Bra,A lower cut makes this supersoft bra perfect f...,bras,50.617021,50.617021,1.535736,0.311891,0.00


In [3]:
victoria_secret.drop('Unnamed: 0', inplace= True, axis = 1)

In [4]:
victoria_secret['description'] = victoria_secret['description'].str.lower()
victoria_secret.head()

,product_name,product_category,description,product_category_group,mrp,price,size_popularity,color_popularity,%discount
0,Allover Lace from Cotton Lingerie NEW! Dotted ...,Dotted Mesh Bikini Panty,dotted mesh in the ultimate everyday shape mak...,panties,10.500000,10.500000,0.126413,0.154872,0.00
1,Allover Lace from Cotton Lingerie NEW! Dotted ...,Dotted Mesh Thong Panty,dotted mesh in a sexy little shape makes this ...,panties,10.500000,10.500000,0.126413,0.154872,0.00
2,Body by Victoria Cheekini Panty,Cheekini Panty,"the comfiest way to show a little cheek peek, ...",panties,14.500000,7.142542,0.754805,0.163567,50.74
3,Body by Victoria Daisy Lace Slip,Daisy Lace Slip,what dreams are made of: this little slip in f...,slip,52.000000,34.990000,0.126413,0.155092,32.71
4,Body by Victoria Demi Bra,Demi Bra,a lower cut makes this supersoft bra perfect f...,bras,50.617021,50.617021,1.535736,0.311891,0.00


In [5]:
# Stopwords removal
stopwords = [ "a", "about", "above", "after", "again", "against", "all", "am", "an", "and", "any", "are", "as", "at", "be", "because", 
             "been", "before", "being", "below", "between", "both", "but", "by", "could", "did", "do", "does", "doing", "down", "during",
             "each", "few", "for", "from", "further", "had", "has", "have", "having", "he", "he'd", "he'll", "he's", "her", "here", 
             "here's", "hers", "herself", "him", "himself", "his", "how", "how's", "i", "i'd", "i'll", "i'm", "i've", "if", "in", "into",
             "is", "it", "it's", "its", "itself", "let's", "me", "more", "most", "my", "myself", "nor", "of", "on", "once", "only", "or",
             "other", "ought", "our", "ours", "ourselves", "out", "over", "own", "same", "she", "she'd", "she'll", "she's", "should", 
             "so", "some", "such", "than", "that", "that's", "the", "their", "theirs", "them", "themselves", "then", "there", "there's",
             "these", "they", "they'd", "they'll", "they're", "they've", "this", "those", "through", "to", "too", "under", "until", "up",
             "very", "was", "we", "we'd", "we'll", "we're", "we've", "were", "what", "what's", "when", "when's", "where", "where's",
             "which", "while", "who", "who's", "whom", "why", "why's", "with", "would", "you", "you'd", "you'll", "you're", "you've",
             "your", "yours", "yourself", "yourselves" ]

In [6]:
import string

In [7]:
def remove_stopwords(data):
  data['clean_description'] = data['description'].apply(lambda x : ' '.join([word for word in x.split() if word not in (stopwords)]))
  return data

vs_clean = remove_stopwords(victoria_secret)
vs_clean['clean_description'] = vs_clean['clean_description'].str.replace('[{}]'.format(string.punctuation), ' ')
vs_clean

,product_name,product_category,description,product_category_group,mrp,price,size_popularity,color_popularity,%discount,clean_description
0,Allover Lace from Cotton Lingerie NEW! Dotted ...,Dotted Mesh Bikini Panty,dotted mesh in the ultimate everyday shape mak...,panties,10.500000,10.500000,0.126413,0.154872,0.00,dotted mesh ultimate everyday shape makes pret...
1,Allover Lace from Cotton Lingerie NEW! Dotted ...,Dotted Mesh Thong Panty,dotted mesh in a sexy little shape makes this ...,panties,10.500000,10.500000,0.126413,0.154872,0.00,dotted mesh sexy little shape makes pretty tho...
2,Body by Victoria Cheekini Panty,Cheekini Panty,"the comfiest way to show a little cheek peek, ...",panties,14.500000,7.142542,0.754805,0.163567,50.74,comfiest way show little cheek peek panty sup...
3,Body by Victoria Daisy Lace Slip,Daisy Lace Slip,what dreams are made of: this little slip in f...,slip,52.000000,34.990000,0.126413,0.155092,32.71,dreams made of little slip flirty daisy lace ...
4,Body by Victoria Demi Bra,Demi Bra,a lower cut makes this supersoft bra perfect f...,bras,50.617021,50.617021,1.535736,0.311891,0.00,lower cut makes supersoft bra perfect revealin...
...,...,...,...,...,...,...,...,...,...,...
664,Victoria Sport The Player by Victoria Sport Sp...,The Player by Victoria Sport Sport Bra,"from cycling to circuit training, this medium-...",bras,16.710526,16.710526,0.478519,0.052107,0.00,cycling circuit training medium support sport...
665,Victoria Sport The Player by Victoria SportLac...,The Player by Victoria SportLace-Up Sport Bra,seamless comfort and a strappy back make this ...,bras,24.500000,24.500000,0.252651,0.087351,0.00,seamless comfort strappy back make medium supp...
666,Victoria Sport The Player by Victoria Sport La...,The Player by Victoria Sport Lace-Up Sport Bra,seamless comfort and a strappy back make this ...,bras,24.500000,24.500000,0.250512,0.094717,0.00,seamless comfort strappy back make medium supp...
667,Victoria Sport The Player by Victoria Sport Pl...,The Player by Victoria Sport Plunge Sport Bra,"pro-support, anti-smash: this sport bra teams ...",bras,24.500000,24.500000,0.252826,0.119766,0.00,pro support anti smash sport bra teams plung...


In [8]:
# from keras.preprocessing.text import Tokenizer
# from keras.layers import Embedding

# tokenizer = Tokenizer(num_words=5000)
# tokenizer.fit_on_texts(vs_clean['clean_description'])
# words_to_index = tokenizer.word_index

# def read_glove_vector(glove_vec):
#   with open(glove_vec, 'r', encoding='UTF-8') as f:
#     words = set()
#     word_to_vec_map = {}
#     for line in f:
#       w_line = line.split()
#       curr_word = w_line[0]
#       word_to_vec_map[curr_word] = np.array(w_line[1:], dtype=np.float64)
#   return word_to_vec_map

# word_to_vec_map = read_glove_vector('Dataset/Glove/glove.6B/glove.6B.50d.txt')
# maxLen = 150
# vocab_len = len(words_to_index)
# embed_vector_len = word_to_vec_map['moon'].shape[0]
# emb_matrix = np.zeros((vocab_len, embed_vector_len))

# for word, index in words_to_index.items():
#   embedding_vector = word_to_vec_map.get(word)
#   if embedding_vector is not None:
#     emb_matrix[index, :] = embedding_vector

# embedding_layer = Embedding(input_dim=vocab_len, output_dim=embed_vector_len, input_length=maxLen, weights = [emb_matrix], trainable=False)

In [9]:
from keras.preprocessing.text import Tokenizer

raw_text = np.hstack([vs_clean.clean_description.str.lower()])
tok_raw = Tokenizer()
tok_raw.fit_on_texts(raw_text)
vs_clean["seq_description"] = tok_raw.texts_to_sequences(vs_clean.clean_description.str.lower())
max_seq_description = np.max(vs_clean.seq_description.apply(lambda x: len(x)))
vs_clean.head()

,product_name,product_category,description,product_category_group,mrp,price,size_popularity,color_popularity,%discount,clean_description,seq_description
0,Allover Lace from Cotton Lingerie NEW! Dotted ...,Dotted Mesh Bikini Panty,dotted mesh in the ultimate everyday shape mak...,panties,10.500000,10.500000,0.126413,0.154872,0.00,dotted mesh ultimate everyday shape makes pret...,"[354, 45, 281, 140, 22, 142, 44, 86, 227, 243,..."
1,Allover Lace from Cotton Lingerie NEW! Dotted ...,Dotted Mesh Thong Panty,dotted mesh in a sexy little shape makes this ...,panties,10.500000,10.500000,0.126413,0.154872,0.00,dotted mesh sexy little shape makes pretty tho...,"[354, 45, 49, 63, 22, 142, 44, 92, 740, 146, 1..."
2,Body by Victoria Cheekini Panty,Cheekini Panty,"the comfiest way to show a little cheek peek, ...",panties,14.500000,7.142542,0.754805,0.163567,50.74,comfiest way show little cheek peek panty sup...,"[933, 119, 65, 63, 98, 105, 86, 61, 185, 186, ..."
3,Body by Victoria Daisy Lace Slip,Daisy Lace Slip,what dreams are made of: this little slip in f...,slip,52.000000,34.990000,0.126413,0.155092,32.71,dreams made of little slip flirty daisy lace ...,"[634, 228, 635, 63, 190, 146, 282, 3, 248, 116..."
4,Body by Victoria Demi Bra,Demi Bra,a lower cut makes this supersoft bra perfect f...,bras,50.617021,50.617021,1.535736,0.311891,0.00,lower cut makes supersoft bra perfect revealin...,"[372, 83, 142, 61, 42, 35, 283, 295, 261, 168,..."


In [14]:
from keras.preprocessing.sequence import pad_sequences

def get_keras_data(dataset):
    X = {
        'item_desc': pad_sequences(dataset.seq_description, maxlen=75)
    }
    return X
X = get_keras_data(vs_clean)

ImportError: cannot import name 'pad_sequences' from 'keras.preprocessing.sequence' (C:\Users\user\anaconda3\lib\site-packages\keras\preprocessing\sequence.py)